In [184]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import csv
from selenium.webdriver import Chrome

In [140]:

base_url = 'https://www.tripadvisor.co.kr/'

In [376]:
# 관광지의 이름 및 link
# 한페이지당 30
title_links = []
title_list = []
image_urls = []
info_list = []

browser = Chrome('./chromedriver.exe')
url = 'https://www.tripadvisor.co.kr/Attractions-g187070-Activities-France.html'
browser.get(url)
browser.implicitly_wait(0.5)
for page in range(0,1):
    if page != 0:
        p = str(page*30)
        url = f'https://www.tripadvisor.co.kr/Attractions-g187070-Activities-oa{p}-France.html'
        browser.get(url)
        browser.implicitly_wait(0.5)
        
    title_url_selector = browser.find_elements_by_css_selector('div._3LwzvfhX > div._25PvF8uO > div > div._6sUF3jUd > a._1QKQOve4')
    
    title_urls = [title.get_attribute('href') for title in title_url_selector]
    titles = [title.get_attribute('text') for title in title_url_selector]
    
    title_links.extend(title_urls)
    title_list.extend(titles)
    
    
for link in title_links[:10]:
    browser.get(link)
    time.sleep(1)
    image_selector = browser.find_element_by_css_selector('div._39avNmow > div > div > ul > li > div > img')
    info_selector = browser.find_element_by_css_selector('div._3RTCF0T0 > span > a')
    
    info_tag = info_selector.get_attribute('text')
    image_tag = image_selector.get_attribute('src')
    
    info_list.append(info_tag)
    image_urls.append(image_tag)


In [344]:
for link in title_links[:]:
    browser.get(link)
    time.sleep(1)
    image_selector = browser.find_element_by_css_selector('div._39avNmow > div > div > ul > li > div > img')
    info_selector = browser.find_element_by_css_selector('div._3RTCF0T0 > span > a')
    
    info_tag = info_selector.get_attribute('text')
    image_tag = image_selector.get_attribute('src')
    
    info_list.append(info_tag)
    image_urls.append(image_tag)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._39avNmow > div > div > ul > li > div > img"}
  (Session info: chrome=85.0.4183.121)


In [377]:
image_urls

['https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/26/b3/6e/20180601-122841-largejpg.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/03/f5/ea/a5/vieille-ville-ciudad.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/f0/0d/d2/photo1jpg.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/da/01/47/vieux-lyon.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0c/60/4e/b2/photo0jpg.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/e0/3b/19/photo0jpg.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/0f/58/b9/d0/by-night.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/e0/7f/43/magnifique-plage-a-l.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/10/8a/a5/9e/che-panorama-da-lassu.jpg?w=800&h=700&s=1',
 'https://dynamic-media-cdn.tripa

In [378]:
info_list

['아트뮤지엄',
 '인근 지역',
 '신성한 & 종교적인 장소',
 '유서 깊은 산책로',
 '즐길거리 & 랜드마크',
 '인근 지역',
 '즐길거리 & 랜드마크',
 '해변',
 '전망대',
 '디즈니 파크 & 액티비티']

In [379]:
title_list

['오르세 미술관',
 'Old Town',
 'Basilique Notre Dame de la Garde',
 'Vieux Lyon',
 'Place de la Bourse',
 'La Petite France',
 'Le Capitole',
 'Plage de la Grande Conque',
 '캐슬 힐',
 '디즈니랜드 파크',
 'Vieux-Lille',
 '월트 디즈니 스튜디오 파크',
 'Iles de Lerins',
 '\xa0대성당 노트르담 드 스트라스부르',
 'Jardin des Plantes',
 'Musee Fabre',
 'Aiguille du Midi',
 'Aquarium La Rochelle',
 'Lake Annecy',
 'Gorges de la Restonica',
 "Le Sentier du Littoral, Cap d'Antibes",
 "Val d'Europe",
 'Hotel de Caumont - Art Centre',
 'Cathedrale Notre-Dame de Reims',
 'The Hall of Mirrors',
 'The Water Mirror',
 'La Croisette',
 'Old Town',
 'Parc National des Calanques',
 'Palais des Papes']

In [380]:
# 여행지 기본정보
f1 = open('France_1.csv', 'w', encoding='utf-8-sig',  newline='')
country = '프랑스'
wr = csv.writer(f1)
wr.writerow(['country','sight','image_url','info'])
for sight, url, info in zip(title_list[:10], image_urls, info_list):
    wr.writerow([country, sight, url, info])
f1.close()

In [381]:
# 가져온 url로 들어가서 체크박스 체크 후 더보기 버튼을 클릭하고 리뷰를 가져온다.
# 보통, 별로, 최악 체크박스 체크

def get_review_data(review_titles, review_contents): 
    # 만약 가져오는 데이터가 없으면 0을 반환 아니면 1을 반환
    isend_st = len(review_titles)
    
    review_title_sel = browser.find_elements_by_css_selector('.ocfR3SKN')
    for rt in review_title_sel:
        print(rt.text, '\n')
        review_titles.append(rt.text)

    isend_end = len(review_titles)
    if isend_st == isend_end:
        return 0

    reviews = browser.find_elements_by_css_selector('.IRsGHoPm')
    for review in reviews:
        print(review.text,'\n')
        review_contents.append(review.text)
    return 1

        
def get_review_data_list(title_link, title):
    
    browser.get(title_link)
    time.sleep(2)
#     checkbox_normal = browser.find_element_by_xpath('//*[@id="component_18"]/div[3]/div/div[1]/div[1]/div[1]/ul/li[3]/label').click() 
    checkbox_worse = browser.find_element_by_css_selector('ul._2lcHrbTn > li:nth-child(4) > label').click()
    checkbox_worst = browser.find_element_by_css_selector('ul._2lcHrbTn > li:nth-child(5) > label').click()
    time.sleep(2)
    review_titles = []
    review_contents = []
    # 첫 페이지 
    get_review_data(review_titles, review_contents) 
    # 리뷰 다음페이지 있으면 넘어가기 최대 20페이지까지
    if browser.find_elements_by_css_selector('.pageNumbers') == []:
        return review_titles, review_contents
    
    next = browser.find_elements_by_css_selector('.pageNumbers > a')
    
    if len(next) != 0:
        last_page = next[-1].text
        # 두번째 url
        sec = next[0].get_attribute('href').split('Reviews')

        if int(last_page) < 5:
            for i in range(2, int(last_page)+1):
                url = sec[0] + 'Reviews-or' + str((i-1)*5) + '-' + sec[1]
                browser.get(url)
                time.sleep(2)
                is_end = get_review_data(review_titles, review_contents)
                # 더이상 리뷰가 없으면 함수 종료
                if is_end == 0:
                    return review_titles, review_contents
                

        else:
            for i in range(2, 6):
                url = sec[0] + 'Reviews-or' + str((i-1)*5) + '-' + sec[1]
                browser.get(url)
                time.sleep(2)
                is_end = get_review_data(review_titles, review_contents)
                # 더이상 리뷰가 없으면 함수 종료
                if is_end == 0:
                    return review_titles, review_contents
                
        
                
    return review_titles, review_contents

In [244]:
if browser.find_elements_by_css_selector('.pageNumbers') == []:
    print("dd")

dd


In [382]:
# 10개의 여행지 하나당 최대 25개의 리뷰를 가져온다.
country = '프랑스'
browser = Chrome('./chromedriver.exe')
f = open('France.csv', 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['country','sight','review_title','review_content'])

for link, tit in zip(title_links[:10], title_list[:10]):
    review_titles, review_contents = get_review_data_list(link, tit)
    for rt, rc in zip(review_titles, review_contents):
        wr.writerow([country, tit ,rt , rc])

f.close()
    

돈을 낭비하지 마라. 

오랜 기다림, 무례한 직원 

한 일부, 아니나 

아주 불만족 스럽 습니다 

정말 말도 안되는 줄 

이 장소를위한 선은 거대했다. 그래서 우리는 더 빨리 들어가기 위해 외부의 온라인을 샀다. 우리가 귀찮게하지 않았 으면 좋겠어. 이 장소에 관한 유일한 좋은 점은 장소입니다. 옛날 개조 한 기차역. 직원 무례하고 도움이되지. 예술을 보는 순서가 없습니다. 목적없이 방황하는 사람들. 루브르 박물관이 예약 되었기 때문에 여기에갔습니다. 방금 밖에서 거리 공연자를 봤어야 했어. 더 싸고 더 재미 있었을까요? 

한 시간 정도를 기다리고 직원들은 매우 무례 했습니다. 좋은 박물관 이었지만 전체적으로 즐거운 경험 였습니다. 

미술관 전체가 큰 이름, 작은 초만원 및 압도적. 분위기는 피곤 했습니다. 많은 보안 검사기. 

이 좋은 것은 가장 중요한 이 박물관은 미술 작품 을 보고 그 범주 - - 그렇지 않으면 때문에 할 수 없을 곳이 포인트. 레이아웃은 끔찍한, " 새로운 " 인상파 의 바닥은 재해 복구 아키텍처, 모든 벽이 검은 색으로 칠해진 말할 것 - - 정말 좋은 색상들은 뒤에 있어, 화려한 그림들로. 경비원들이 돌아 다니는, 대부분의 휴대 전화에서 찾고. 아이들은 본 장소를 모두 실행하거나 부모 말도 되지 않습니다. 가 별로 다. 예술 하면 가 본 적이 없습니다. 할 경우, 귀마개를 가지고 이해를 

구입 한 티켓 을 미리 보는 큐에 박물관 입니다. 하지만, 그럼에도 불구하고 고급 티켓까지 30분 대한 외부 큐 추운 날씨. 유지되지만 헛수고 의 잘못 을 검색한 온라인 티켓을 구입 해 줍니다. 이 박물관 의 가장 큰 이유인. 일단 내부는 놀라운 건물 이 멋진 예술 작품을 볼 수 있는. 하지만, 또 음식에 세 줄을 화장실에 카페까지. 예술 은 인상파 와 가게에서 스크럼. 조용하고 평화로운 예술 작품을 보는, 이 곳은 아닙니다. 

현대적인 

끔찍한 낭비 

너무 많은 사람들이 

실망... 

폐쇄 예고 없이! 

이 박물관은 루브

워터 미러가 없습니까? 

빈 공간 

보르도 전체가 과대 평가되었지만 아무도 그것을 말할만큼 용감하지 않습니다. 

반사 없음 

과장 가치가 없다. 

워터 미러가 작동하지 않으면 볼 것이 없습니다. 일련의 형상 건물입니다. 불일치. 

모두가 여기에서 만나 모든 가이드 북에 있지만 분수대 외에는 볼거리가 없습니다. 주요 도시로 빠르게 출입 할 수 있습니다. 

20 년 이상 보르도 방문. 그 때 덤프 였고 여전히 그렇습니다! 전체 도시가 파다. 도처에 쓰레기가 있고, 포장 도로와 잡초에 잡초가 있습니다. 육군은 12 개 화장실을 통제하고 70 세 이상은 출입구와 엄마들에게 거짓말을하며 12시에 아기들을 태우고 아기를 태우고있다. 그들은 사회 복지 사업을 가지고 있지 않습니까? 

나는 주말 내내 보르도에 있었고이 거울상을 한 번 보지 못했다. 비록 주말을 위해 떨어져 있더라도 관광객에게 알리기 위하여 아무 고시도 없다. 

이것이 왜 그렇게 매력적인지 나는 알 수 없다. 우리가 11 월에 거기에 있었고 건물의 절반에 걸쳐 비계가 있었지만, 나는 과대 광고가 무엇인지 전혀 알지 못한다. 이 목록에서 쉽게 빠질 수 있습니다. 미안 해요. 

작은 아이들의 놀이터 

실망 

Humph! 

좋은 물은 아이들에게 따뜻한 일. 사람이 붐비 사용자_이름:. 떨어지지 않는 마케팅의 사진 작업이 이 미러, 조용한 충분한 효과는 물이 되지 않습니다. 

사진보다 훨씬 더 흥미롭고 아름다운 왕궁 의 진짜. 저희가 방문한 날도 햇살. 

실망스럽게도 이 아닌 가치가 있다. 곡선 발판 및 설정과 함께 적용하는 임시 이여폰에 트럭 훼이셔, "미러" 이 작업을 통해 튀는 아이들이 헤아릴 수 없이 많은 수 그냥하지 않습니다. 이 곳의 과대 광고를 믿지마세요! 

지역 주민들에게는 아이디어가 없습니다. 

그다지 리뷰 

하지? 

그냥 봉제어능 은 산책 

결코 그것을 발견하지 못하여 그것을 재검토 할 수 없다. 우리는 그것을 찾기 위해 주위에 탔다, 우리지도를 보았다, Goog

In [383]:
browser = Chrome('./chromedriver.exe')
f = open('France.csv', 'a', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
# wr.writerow(['country','sight','review_title','review_content'])

for link, tit in zip(title_links[10:11], title_list[10:11]):
    review_titles, review_contents = get_review_data_list(link, tit)
    for rt, rc in zip(review_titles, review_contents):
        wr.writerow([country, tit ,rt , rc])

f.close()

"구"에 릴 방문 

실망스러운 

우리는 함께 해서 실망스러웠습니다 " 올드 타운 " 릴. 이 있지만 멋진 오래된 건물을 방문 했을 때, 이 비계 가 공사 중 또는 광고판 에 가지지 있습니다. 이 중 특히 그랜드 플레이스, 센터는 전체적으로 숨겨진 게 없고 분수에 키가 큰 광고판. 전반적인 인상 였는데 이 모든 것을 아무도 돌봐 주지 않는 것 같았다. 꽤 초라한 거리, 많은 것이다. 우리는 좋은 가게들이 많은 생각하지 않았다. 사실 일요일 이었기 때문에 많은 곳이 문을 닫아서, 가게가 오픈 하지 않고 일요일에 프랑스의. 우리는 

아, 하지만 전체적으로 좋은 건물이 있고 실망스러웠습니다. 릴 것은 너무 예쁜 도시, 세계에서 멀리 떨어져 있고, 말하는 것, 할 것. 다른 리뷰들을 보고 놀랐다. 또한 마약 중독자 의 수는 유형 및 소지품 걸어나갈 때를 오히려 eyieng 불미스러운 불쾌 해. 



In [366]:
f.close()

In [298]:
for link, tit in enumerate(zip(title_links[:], title_list[:])):
    print(i, link, tit)

11 0 ('https://www.tripadvisor.co.kr/Attraction_Review-g274707-d276251-Reviews-Prague_Zoo-Prague_Bohemia.html', '프라하 동물원')
11 1 ('https://www.tripadvisor.co.kr/Attraction_Review-g274714-d277576-Reviews-Cathedral_of_St_Peter_and_St_Paul-Brno_South_Moravian_Region_Moravia.html', 'Cathedral of St. Peter and St. Paul')
11 2 ('https://www.tripadvisor.co.kr/Attraction_Review-g274707-d275451-Reviews-Staromestske_namesti-Prague_Bohemia.html', '구시가')
11 3 ('https://www.tripadvisor.co.kr/Attraction_Review-g274714-d277581-Reviews-Villa_Tugendhat-Brno_South_Moravian_Region_Moravia.html', 'Villa Tugendhat')
11 4 ('https://www.tripadvisor.co.kr/Attraction_Review-g274697-d276225-Reviews-Diana_Lookout_Tower-Karlovy_Vary_Karlovy_Vary_Region_Bohemia.html', 'Diana Lookout Tower')
11 5 ('https://www.tripadvisor.co.kr/Attraction_Review-g274714-d277575-Reviews-Spilberk_Castle-Brno_South_Moravian_Region_Moravia.html', 'Spilberk Castle')
11 6 ('https://www.tripadvisor.co.kr/Attraction_Review-g274707-d275157-R